In [1]:
# !pip install nltk
# !pip install Sastrawi
# nltk.download('punkt')
# nltk.download('stopwords')

In [2]:
import re
import string
import numpy as np
import pandas as pd
from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk.probability import FreqDist
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

df_labeled = pd.read_csv('data_labeled.csv')
# df_labeled.head()
df_labeled.shape

(546, 2)

In [3]:
def cleansing(text):
    text = ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)"," ",text).split()) #rm hashtag, link, mention
    text = text.replace('\\t'," ").replace('\\n'," ").replace('\\u'," ").replace('\\',"")  #rm tab, new line, and back slice
    text = text.encode('ascii', 'replace').decode('ascii')  #rm non ASCII (emoticon, chinese word, .etc)
    text = re.sub(r"\d+","", text) #rm number
    text = re.sub('\s+'," ", text) #rm multiple space
    text = "".join([char for char in text if char not in string.punctuation]) #rm puctuation
    return text
    
def case_folding(text):
    text = text.lower()
    return text
    
def tokenization(text):
    token = word_tokenize(text)
    text = token
    return text

def stopword_removal(text):
    list_stopwords = stopwords.words('indonesian')
    txt_stopword = pd.read_csv("add_stopwords.txt", names= ["stopwords"], header = None)
    list_stopwords.extend(txt_stopword["stopwords"][0].split(' '))
    text = [word for word in text if word not in list_stopwords]
    return text

def normalization(text):
    normalizad_word_dict = {}
    normalizad_word = pd.read_excel("add_normalization.xlsx")
    for index, row in normalizad_word.iterrows():
        if row[0] not in normalizad_word_dict:
            normalizad_word_dict[row[0]] = row[1]  
    text = [normalizad_word_dict[word] if word in normalizad_word_dict else word for word in text]
    return text

def stemming(text):
    factory = StemmerFactory()
    stemmer = factory.create_stemmer()
    text = [stemmer.stem(word) for word in text]
    return text
    
df_preprocess = df_labeled
df_preprocess['cleansing'] = df_preprocess['comments'].apply(cleansing)
df_preprocess['casefolding'] = df_preprocess['cleansing'].apply(case_folding)
df_preprocess['tokenization'] = df_preprocess['casefolding'].apply(tokenization)
df_preprocess['stopword_removal'] = df_preprocess['tokenization'].apply(stopword_removal)
df_preprocess['normalization'] = df_preprocess['stopword_removal'].apply(normalization)
df_preprocess['stemming'] = df_preprocess['normalization'].apply(stemming)

df_preprocessed = df_preprocess

In [22]:
target = []
for index, row in df_preprocessed.iterrows():
    if row["label"] == "positif":
        target.append(1)
    else:
        target.append(0)
        
df_preprocessed['target'] = target
df_preprocessed.head()

,comments,label,cleansing,casefolding,tokenization,stopword_removal,normalization,stemming,target
0,"BBM NAIK, BANDINGKAN HARGANYA DENGAN SINGAPURA, MALAYSIA DAN BRUNEI-&gt; <a href=""https://wow.tribunnews.com/2022/09/03/bbm-naik-berikut-perbandingan-harga-di-indonesia-dengan-singapura-malaysia-dan-brunei"">https://wow.tribunnews.com/2022/09/03/bbm-naik-berikut-perbandingan-harga-di-indonesia-dengan-singapura-malaysia-dan-brunei</a>",negatif,BBM NAIK BANDINGKAN HARGANYA DENGAN SINGAPURA MALAYSIA DAN BRUNEI gt a href,bbm naik bandingkan harganya dengan singapura malaysia dan brunei gt a href,"[bbm, naik, bandingkan, harganya, dengan, singapura, malaysia, dan, brunei, gt, a, href]","[bbm, bandingkan, harganya, singapura, malaysia, brunei]","[bbm, bandingkan, harganya, singapura, malaysia, brunei]","[bbm, banding, harga, singapura, malaysia, brunei]",0
1,Seng setuju mundak berarti goblok.<br>Jokowi kongkon melek.<br>Ngoros BBM isok malah blt tepat sasaran.,negatif,Seng setuju mundak berarti goblok br Jokowi kongkon melek br Ngoros BBM isok malah blt tepat sasaran,seng setuju mundak berarti goblok br jokowi kongkon melek br ngoros bbm isok malah blt tepat sasaran,"[seng, setuju, mundak, berarti, goblok, br, jokowi, kongkon, melek, br, ngoros, bbm, isok, malah, blt, tepat, sasaran]","[seng, setuju, mundak, goblok, jokowi, kongkon, melek, ngoros, bbm, isok, blt, sasaran]","[kalau, setuju, bangkit, goblok, jokowi, terkirim, terpelajar, konsumsi, bbm, besok, blt, sasaran]","[kalau, tuju, bangkit, goblok, jokowi, kirim, ajar, konsumsi, bbm, besok, blt, sasar]",0
2,"jokes standup kaka lucu sekali,<br> lapangan pekerjaan sama pendapatan masyarakatnya gak dilihat... 😭😭😭😭",negatif,jokes standup kaka lucu sekali br lapangan pekerjaan sama pendapatan masyarakatnya gak dilihat,jokes standup kaka lucu sekali br lapangan pekerjaan sama pendapatan masyarakatnya gak dilihat,"[jokes, standup, kaka, lucu, sekali, br, lapangan, pekerjaan, sama, pendapatan, masyarakatnya, gak, dilihat]","[jokes, standup, kaka, lucu, lapangan, pekerjaan, pendapatan, masyarakatnya, gak]","[canda, standup, kakak, lucu, lapangan, pekerjaan, pendapatan, masyarakatnya, tidak]","[canda, standup, kakak, lucu, lapang, kerja, dapat, masyarakat, tidak]",0
3,Kok yo dibanding bandike sama UMR besar woy,negatif,Kok yo dibanding bandike sama UMR besar woy,kok yo dibanding bandike sama umr besar woy,"[kok, yo, dibanding, bandike, sama, umr, besar, woy]","[yo, dibanding, bandike]","[ya, dibanding, banding]","[ya, banding, banding]",0
4,Gagal total jokowi,negatif,Gagal total jokowi,gagal total jokowi,"[gagal, total, jokowi]","[gagal, total, jokowi]","[gagal, total, jokowi]","[gagal, total, jokowi]",0


In [5]:
print('Target:')
print(df_preprocessed["target"].value_counts())

Target:
0    375
1    171
Name: target, dtype: int64


In [6]:
samplePositive = df_preprocessed[df_preprocess['target']==1].sample(171, replace=True)
sampleNegative = df_preprocessed[df_preprocess['target']==0].sample(171, replace=True)

df_preprocessed_sample = pd.concat([samplePositive, sampleNegative])
df_preprocessed_sample = df_preprocessed_sample.iloc[:, [7,8]]

df_preprocessed_origin = df_preprocessed.iloc[:, [7,8]]

In [7]:
# df_preprocessed_origin.to_csv('data_preprocessed_origin.csv', index=False)
# df_preprocessed_sample.to_csv('data_preprocessed_sample.csv', index=False)

In [8]:
# df_preprocessed.to_excel('data_preprocessed.xlsx', index=False)

In [9]:
# df_preprocessed_sample.to_csv('data_preprocessed_sample_15-06-2023.csv', index=False)

In [10]:
pd.set_option('display.max_colwidth', None)

In [23]:
df_preprocessed.loc[[236], ['comments','cleansing','casefolding','tokenization','stopword_removal','normalization', 'stemming']]

,comments,cleansing,casefolding,tokenization,stopword_removal,normalization,stemming
236,"Rakyat pada protes bbm naik karna selain harga bbmnya semakin mahal.sembako,dll juga pada naik harganya kalo bbm naik(biasanya).",Rakyat pada protes bbm naik karna selain harga bbmnya semakin mahal sembako dll juga pada naik harganya kalo bbm naik biasanya,rakyat pada protes bbm naik karna selain harga bbmnya semakin mahal sembako dll juga pada naik harganya kalo bbm naik biasanya,"[rakyat, pada, protes, bbm, naik, karna, selain, harga, bbmnya, semakin, mahal, sembako, dll, juga, pada, naik, harganya, kalo, bbm, naik, biasanya]","[rakyat, protes, bbm, karna, harga, bbmnya, mahal, sembako, dll, harganya, kalo, bbm]","[rakyat, protes, bbm, karena, harga, bbmnya, mahal, sembako, lainnya, harganya, kalau, bbm]","[rakyat, protes, bbm, karena, harga, bbmnya, mahal, sembako, lain, harga, kalau, bbm]"
